<a href="https://colab.research.google.com/github/forte-heck-rum/test/blob/master/detect_racial_slurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required packages

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

%matplotlib inline

Data:

In [4]:
train = pd.read_csv('train.csv')
print("Training Set:"% train.columns, train.shape, len(train))
test = pd.read_csv('test.csv')
print("Test Set:"% test.columns, test.shape, len(test))

Training Set: (31962, 3) 31962
Test Set: (17197, 2) 17197


Cleaning:

In [5]:
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

Upsampling:

In [9]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

TF-IDF, Count Vectorizer, SGD Pipeline:

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),
])

Splitting training and testing model:

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],                                    train_upsampled['label'],random_state = 0)

Accuracy: (f1 score)

In [12]:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.969729297239632

Final table:

In [49]:
data = X_test
df = pd.DataFrame(data)
df['label'] = y_predict
hate =[]
for i in y_predict:
  if i==0:
    hate.append('no')
  elif i==1:
    hate.append('yes')
df['hateful/racial y/n'] = hate
df

,tweet,label,hateful/racial y/n
1095,bihday brother cool thebomb,0,no
9766,gf asked me to make an account i told her i di...,0,no
26909,sunday withyou happiness family vinallop,0,no
1386,been feeling low for ages and when the one per...,0,no
4616,chaplin the dictator speech via theresista...,1,yes
...,...,...,...
17825,im looking forward to a few titles being annou...,0,no
30707,lol these pasty cakes telling the rockette to ...,1,yes
15560,gorgeous evening family lakesimcoe willow b...,0,no
30566,rain or shine this treasure hunt is gonna go d...,0,no
